In [1]:
import requests
import json
import base64   #codiert binärdateien zu text (ich brauche ja was aus der readme-datei)
import re   # python modul für reguläre ausdrücke (zb Überschriften)

In [2]:
def get_readme_heading(owner, repo):   # funktion mit 2 argumenten: Benutzer, name des repos
    url = f"https://api.github.com/repos/{owner}/{repo}/readme"  #fragt readme-datei ab
    headers = {"Accept": "application/vnd.github.v3+json"}  # sagt, dass ich das in json haben will

    r = requests.get(url, headers=headers) # abrufen der readme-datei, ergebnis ist ein response-objekt

    if r.status_code == 200:   #prüft, ob Anfrage erfolgreich war (200 heißt erfolgreich)
        data = r.json()
        content_encoded = data.get("content", "")   # holt content-Feld (base64-codierter string)
        content = base64.b64decode(content_encoded).decode("utf-8", errors="ignore")  # verwandelt codierte zeichen in normal lesbaren Text

        match = re.search(r"^#\s+(.+)", content, re.MULTILINE)   # suche nach der ersten Zeile, die mit # beginnt, also einer H1-Überschrift in Markdown.
        if match:
            return match.group(1).strip()   # der Text nach dem #
        else:
            return "keine Überschrift gefunden"
    else:
        return f"kein README ({r.status_code})"

# vorher definierte Funktion auf df_gitgu_sw anwenden, owner+repo benötigt -> full_name
df_gitgu_sw["readme_überschrift"] = df_gitgu_sw["full name"].apply(lambda full_name: get_readme_heading(*full_name.split("/")))
df_gitgu_sw